In [1]:
from pymongo import MongoClient
from datetime import datetime

MONGO_CONFIG = {
    'uri': "mongodb://ich_editor:verystrongpassword"
          "@mongo.itcareerhub.de/?readPreference=primary"
          "&ssl=false&authMechanism=DEFAULT&authSource=ich_edit",
    'database': 'ich_edit',
    'collection': 'project_110425_Noname'
}

def log_search_query(search_type, params, results_count):
    # Настройка подключения к MongoDB
    try:
        mongo_client = MongoClient(MONGO_CONFIG['uri'])
        # mongo_client.admin.command('ping')
        # print('Успешно')
        search_collection = mongo_client[MONGO_CONFIG['database']][MONGO_CONFIG['collection']]
        log_entry = {
            "timestamp": datetime.now(),
            "search_type": search_type,
            "params": params,
            "results_count": results_count
        }
        search_collection.insert_one(log_entry)
        
    except ConnectionFailure as e:
        print(f"Ошибка записи лога: {e}")
    finally:
        mongo_client.close()

In [2]:
log_search_query('keywords', {"keyword": "matrix"}, 3)

Успешно


In [ ]:
Исходные данные
|
Группировка
    |
Сортировка
    |
Ограничение
    |
Результат

In [11]:
def get_most_popular_queries(limit=5):
    mongo_client = MongoClient(MONGO_CONFIG['uri'])
    collection = mongo_client[MONGO_CONFIG['database']][MONGO_CONFIG['collection']]
    # Анализ популярности по типу запросов
    pipeline_type = [
        {"$group": {"_id":"$search_type", "count": {"$sum":1}}},
        {"$sort":  {"count":-1}},
        {"$limit": limit}
    ]
    
    popular_types = list(collection.aggregate(pipeline_type))
    
    # Анализ популярности по ключевым словам (по параметрам)
    pipeline_keywords = [
        {"$match": {"search_type": "keyword"}},
        {"$group": {"_id":"$params.keyword", "count": {"$sum":1}}},
        {"$sort":  {"count":-1}},
        {"$limit": limit}
    ]    
    popular_keywords = list(collection.aggregate(pipeline_keywords))

    # Анализ популярности по жанрам (по параметрам)
    pipeline_genres = [
        {"$match": {"search_type": "genre_year"}},
        {"$group": {"_id":"$params.genre_name", "count": {"$sum":1}}},
        {"$sort":  {"count":-1}},
        {"$limit": limit}
    ]    
    popular_genres = list(collection.aggregate(pipeline_genres))
    
    return {
        "popular_search_types": popular_types,
        "popular_params_keywords": popular_keywords,
        "popular_params_genres": popular_genres
    }

>Два запроса считаются одинаковыми, если у них совпадают И тип И параметры

>Например: `{search_type: "keyword", params: {keyword: "matrix"}}`

Шаг 1. Первоначальная сортировка.

Сортируем все документы в коллекции по убыванию времени.


Шаг 2. Группировка для нахождения уникальных запросов.

Группируем документы по унивальности комбинации search_type + params

In [ ]:
#до
{_id: 2, 15:30, "keyword", {keyword: "matrix"}}
{_id: 1, 15:00, "keyword", {keyword: "matrix"}}
{_id: 3, 14:45, "genre_year", {genre: "Action"}}

In [ ]:
#после
{
  _id: {search_type: "keyword", params: {keyword: "matrix"}},
  latest_timestamp: 15:30,    ← из документа _id: 2
  results_count: 3,           ← из документа _id: 2  
  document_id: 2              ← из документа _id: 2
}
{
  _id: {search_type: "genre_year", params: {genre: "Action"}},
  latest_timestamp: 14:45,    ← из документа _id: 3
  results_count: 5,           ← из документа _id: 3
  document_id: 3              ← из документа _id: 3
}

Шаг 3. Сортировка уникальных запросов

In [15]:
get_most_popular_queries()

{'popular_search_types': [{'_id': 'keyword', 'count': 2},
  {'_id': 'genre_year', 'count': 2},
  {'_id': 'keywords', 'count': 1}],
 'popular_params_keywords': [{'_id': 'matrix', 'count': 1},
  {'_id': 'love and sport', 'count': 1}],
 'popular_params_genres': [{'_id': 'Action', 'count': 2}]}

In [16]:
def get_unique_queries(limit=5):
    mongo_client = MongoClient(MONGO_CONFIG['uri'])
    collection = mongo_client[MONGO_CONFIG['database']][MONGO_CONFIG['collection']]  

    pipeline = [
        {"$sort": {"timestamp": -1}},
        {"$group":{"_id":{"search_type": "$search_type", "params": "$params"},
                  "latest_timestamp":{"$first": "$timestamp"},
                   "results_count": {"$first": "$results_count"},
                   "document_id": {"$first": "_id"}
                  }},
        {"$sort": {"latest_timestamp":-1}},
        {"$limit": limit}
    ]
    unique_queries = list(collection.aggregate(pipeline))
    mongo_client.close()

    return unique_queries

In [17]:
get_unique_queries()

[{'_id': {'search_type': 'keywords', 'params': {'keyword': 'matrix'}},
  'latest_timestamp': datetime.datetime(2025, 9, 30, 10, 39, 33, 953000),
  'results_count': 3,
  'document_id': '_id'},
 {'_id': {'search_type': 'genre_year',
   'params': {'genre_id': 1,
    'genre_name': 'Action',
    'start_year': 2015,
    'end_year': 2015}},
  'latest_timestamp': datetime.datetime(2025, 9, 30, 10, 27, 47, 159000),
  'results_count': 3,
  'document_id': '_id'},
 {'_id': {'search_type': 'genre_year',
   'params': {'genre_id': 1,
    'genre_name': 'Action',
    'start_year': 2005,
    'end_year': 2015}},
  'latest_timestamp': datetime.datetime(2025, 9, 30, 10, 27, 46, 668000),
  'results_count': 3,
  'document_id': '_id'},
 {'_id': {'search_type': 'keyword', 'params': {'keyword': 'love and sport'}},
  'latest_timestamp': datetime.datetime(2025, 9, 30, 10, 27, 46, 141000),
  'results_count': 13,
  'document_id': '_id'},
 {'_id': {'search_type': 'keyword', 'params': {'keyword': 'matrix'}},
  'lates

In [8]:
# 
mongo_client = MongoClient(MONGO_CONFIG['uri'])
search_collection = mongo_client[MONGO_CONFIG['database']][MONGO_CONFIG['collection']]

docs = search_collection.find()
for doc in docs:
    print(doc)

{'_id': ObjectId('68db86710f706f00cae17611'), 'timestamp': datetime.datetime(2025, 9, 30, 10, 27, 45, 488000), 'search_type': 'keyword', 'params': {'keyword': 'matrix'}, 'results_count': 3}
{'_id': ObjectId('68db86720f706f00cae17613'), 'timestamp': datetime.datetime(2025, 9, 30, 10, 27, 46, 141000), 'search_type': 'keyword', 'params': {'keyword': 'love and sport'}, 'results_count': 13}
{'_id': ObjectId('68db86720f706f00cae17615'), 'timestamp': datetime.datetime(2025, 9, 30, 10, 27, 46, 668000), 'search_type': 'genre_year', 'params': {'genre_id': 1, 'genre_name': 'Action', 'start_year': 2005, 'end_year': 2015}, 'results_count': 3}
{'_id': ObjectId('68db86730f706f00cae17617'), 'timestamp': datetime.datetime(2025, 9, 30, 10, 27, 47, 159000), 'search_type': 'genre_year', 'params': {'genre_id': 1, 'genre_name': 'Action', 'start_year': 2015, 'end_year': 2015}, 'results_count': 3}
{'_id': ObjectId('68db89356ddd7199ee5d2dc5'), 'timestamp': datetime.datetime(2025, 9, 30, 10, 39, 33, 953000), 's

```
[{'_id': {'search_type': 'keywords', 'params': {'keyword': 'matrix'}},
  'latest_timestamp': datetime.datetime(2025, 9, 30, 10, 39, 33, 953000),
  'results_count': 3,
  'document_id': '_id'},
 {'_id': {'search_type': 'genre_year',
   'params': {'genre_id': 1,
    'genre_name': 'Action',
    'start_year': 2015,
    'end_year': 2015}},
  'latest_timestamp': datetime.datetime(2025, 9, 30, 10, 27, 47, 159000),
  'results_count': 3,
  'document_id': '_id'},
 {'_id': {'search_type': 'genre_year',
   'params': {'genre_id': 1,
    'genre_name': 'Action',
    'start_year': 2005,
    'end_year': 2015}},
  'latest_timestamp': datetime.datetime(2025, 9, 30, 10, 27, 46, 668000),
  'results_count': 3,
  'document_id': '_id'},
 {'_id': {'search_type': 'keyword', 'params': {'keyword': 'love and sport'}},
  'latest_timestamp': datetime.datetime(2025, 9, 30, 10, 27, 46, 141000),
  'results_count': 13,
  'document_id': '_id'},
 {'_id': {'search_type': 'keyword', 'params': {'keyword': 'matrix'}},
  'latest_timestamp': datetime.datetime(2025, 9, 30, 10, 27, 45, 488000),
  'results_count': 3,
  'document_id': '_id'}]
```

##### Интерфейс

In [27]:
# Популярные запросы
choise = input('Введите нужное действие: 1 -, 2 -, 0 - выход')
if choise == '1':
    #print('Популярные запросы:')
    print('Популярные типы запросов:')
    popular = get_most_popular_queries()
    for item in popular['popular_search_types']:
        print(f'{item['_id']}: {item['count']} запросов')
    
    print('-'*60)
    print('Популярные запросы по параметрам:')
    
    for item in popular['popular_params_keywords']:
        print(f'Ключевое слово: {item['_id']} в кол-ве: {item['count']} запросов')
    
    print('-'*60)
    print('Популярные запросы по жанрам:')
    for item in popular['popular_params_genres']:
        print(f'Жанр: {item['_id']} в кол-ве: {item['count']} запросов')
elif choise == '2':
    popular_unique = get_unique_queries()
    print('Последние уникальные запросы')
    for item in popular_unique:
        print(f'{item['_id']['search_type']} - {item['latest_timestamp']}')
        print(f'Параметры: {item['_id']['params']}')
        print(f'Найдено результатов: {item['results_count']}')
else:
    pass
    # вызов функции главного меню 
    


Введите нужное действие: 1 -, 2 -, 0 - выход 2


Последние уникальные запросы
keywords - 2025-09-30 10:39:33.953000
Параметры: {'keyword': 'matrix'}
Найдено результатов: 3
genre_year - 2025-09-30 10:27:47.159000
Параметры: {'genre_id': 1, 'genre_name': 'Action', 'start_year': 2015, 'end_year': 2015}
Найдено результатов: 3
genre_year - 2025-09-30 10:27:46.668000
Параметры: {'genre_id': 1, 'genre_name': 'Action', 'start_year': 2005, 'end_year': 2015}
Найдено результатов: 3
keyword - 2025-09-30 10:27:46.141000
Параметры: {'keyword': 'love and sport'}
Найдено результатов: 13
keyword - 2025-09-30 10:27:45.488000
Параметры: {'keyword': 'matrix'}
Найдено результатов: 3
